# Privacera Shield protecting LangChain Application
This notebook shows how to use Privacera Shield Library with a LangChain application. To run this notebook you will need the following,


1.  Sign up for a free account at [Privacera AI Governance (PAIG)](https://privacera.ai). This is simple, all you need is your email address.
2.  OpenAI API Key. This will allow you to create your first OpenAI application governed by Privacera AI Governance. Please visit your [OpenAI account](https://platform.openai.com/settings/profile?tab=api-keys) to get this key.

# Step 1 - Install the Python packages
This step will take about a minute to complete.



In [ ]:
!pip install -qqq \
  langchain-community==0.2.0 \
  langchain-openai==0.1.7 \
  langchain==0.2.0 \
  privacera_shield

# Step 2 - Set Up Your Privacera AI Application and Generate API Key
1. Log into your account in PAIG.
2. To create a new application, go to `Paig Navigator > AI Applications` and click the `CREATE APPLICATION` button on the right top. This will open a dialog box where you can enter the details of the application.
3. Navigate to `Paig Navigator -> AI Applications` and select the application you want to generate the api key. In the `API KEYS` tab, click the `GENERATE API KEY` button in the top-right corner to generate an API key. Provide a `Name` and `Description`, along with a `Expiry`.

Note: Once the Api Key is generated, it will not be displayed again. Ensure you copy and securely store it immediately after generation.

# Step 3 - Configure your OpenAI API Key into the environment.
Enter your OpenAI API key in the pop-up. This key is stored in the environment of the notebook and not uploaded to PAIG.

Hit enter after you have pasted the key.

In [ ]:
import os
from getpass import getpass

if os.environ.get("OPENAI_API_KEY") is None:
    openai_api_key = getpass("🔑 Enter your OpenAI API key and hit Enter:")
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Step 4 - Import all the required packages

In [ ]:
import os

from privacera_shield import client as privacera_shield_client
from privacera_shield.exception import AccessControlException
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

# utility function to wrap the output
def wrap_text(text, width=80):
    words = text.split()
    character_count = 0
    for word in words:
        if character_count + len(word) + 1 > width:  # Check if adding the word would exceed the width
            print("\n", end="")  # Start a new line
            character_count = 0  # Reset the character count for the new line
        print(word, end=" ")  # Print the word followed by a space
        character_count += len(word) + 1  # Update the character count
    print()

# Step 5 - Initialize the Privacera Shield library
To initialize the Privacera Shield library in your AI application, set the PAIG_APP_API_KEY either as an environment variable or directly in the code. Choose one of the following methods:

1. Set the PAIG_APP_API_KEY as an environment variable.


<p style="margin-left: 40px;">
  <span style="font-size: 20px;">•</span> Enter the PAIG APP API key obtained from the PAIG portal into the pop-up and press Enter to confirm.
</p>

In [ ]:
import os
from getpass import getpass

if os.environ.get("PAIG_APP_API_KEY") is None:
    paig_app_api_key = getpass("🔑 Enter your PAIG APP API key and hit Enter:")
    os.environ["PAIG_APP_API_KEY"] = paig_app_api_key

# Initialize Privacera Shield
privacera_shield_client.setup(frameworks=["langchain"])

<p style="text-align: center;">
  <span style="font-size: 25px; font-weight: bold;">OR</span>
</p>

2. Pass the API Key in Code
    
    If exporting the PAIG_APP_API_KEY as an environment variable is not preferred, pass it directly as an argument.
    Make Sure to replace API_KEY with the actual apikey generated from paig portal.

In [ ]:
# Initialize Privacera Shield With Provided ApiKey
privacera_shield_client.setup(frameworks=["langchain"], application_config_api_key="API_KEY")

> **Tip:** If you get an error in this step, it could be because you have already run this step once. You can ignore the error and continue. You can also try restarting the Kernel and start from the beginning with the proper JSON file and OpenAI API key.

# Step 6 - Initialize OpenAI based LLM object in LangChain
In this step, we are creating an LLM object for the OpenAI LLM and a prompt template.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")  # (1)

llm = OpenAI(openai_api_key=api_key)
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Step 7 - Run the LLMChain with your question
1. In this step, we are going to run the LLMChain with the prompt asked by a user named `testuser`. 
2. Note how we are passing the username by creating a Privacera Shield context object. 
3. Privacera Shield will intercept the prompt and the response coming from LLM for the `testuser` and run policies. 
4. The PAIG service scans both the prompt and response text and runs security policies. 
5. If the policies decide that the access is denied then an AccessControlException is thrown.

> **Note:** Here username used is `testuser` which is an external user. So the policies applied will be as per the public. For applying userspecific properties, create and use the user from PAIG portal `Settings > Users`.

In [ ]:
# Let's assume the user is "testuser"
user = "testuser"
prompt_text = "Who was the first President of USA and where did they live?"
print(f"Prompt: {prompt_text}")
print()

llm_chain = prompt | llm
try:
    with privacera_shield_client.create_shield_context(username=user):
        response = llm_chain.invoke(prompt_text)
        print(f"LLM Response:")
        wrap_text(response)
except AccessControlException as e:
    # If access is denied, then this exception will be thrown. You can handle it accordingly.
    print(f"AccessControlException: {e}")

# Step 8 - Review the Access Audits in PAIG portal
1. Now you can log in to the PAIG portal and check under `Paig Lens > Audits > Access Audit` tab. Here, You will see the audit record for the above run of your LangChain application.
![title](img/Access_Audits_More_Details.png)
2. You can click on the `More Details` text to view the prompts sent by the application to the LLM and the responses coming from the LLM.
![title](img/Access_Audits_Audit_Content.png)
3. The default policy in PAIG for the application monitors the flow and tags the contents of the prompt and response.

# Step 9 - Modify the policy of your AI application
1. Log into the PAIG portal and navigate to your application under Application section. 
2. Click on the Permissions tab and enable the 'CCPA Personal Information Redaction' policy by clicking on the toggle in the status column. This will start redacting the sensitive data in the Prompt and Reply.
![title](img/AI_App_Policy_Listing.png)
3. Wait for few seconds and re-run Step 7 above. The reply will have the Location and Person name redacted, that is, replaced by `<<LOCATION>>` and `<<PERSON>>`. You can re-run Step 7 a few more times if the above is not redaction is not reflected.
4. Then check the access audits and you will see the original response from the LLM and how Privacera AI redacted the sensitive information out of it.
5. You can also modify the policy by clicking on the pencil icon and change the Prompt to Allow while keeping Reply to Redact.
![title](img/AI_App_Policy_Edit_Restriction.png)
6. Wait for few seconds and re-run Step 7. In the access audits, you will see that the prompt is unchanged while the response is redacted.
7. You can change the question in Step 7 and also try changing the policy to see the effect.